In [ ]:
# y=mx-c model to predict from the data given
# m= 2 and c= 1

import tensorflow as tf
import numpy as np
from tensorflow import keras

model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss='mean_squared_error')

xs=np.array([-1.0, 0.0, 1.0, 2.0], dtype=float)
ys=np.array([-3.0, -1.0, 1.0, 3.0], dtype=float)

model.fit(xs, ys, epochs=1000)
print(model.predict([10.0]))

In [ ]:
# mnist fashion dataset use to find the shoe from the data feed 

import tensorflow as tf
from tensorflow import keras

#make a class callBack to stop the trainning if we got the acurracy upto 95 %
class callback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epochs, logs={}):
        if(logs.get('acc')>0.9):
            print("\n Reached 90% accuracy so stop training the model ")
            self.model.stop_training = True

callbacks=callback()

fashion_minst =tf.keras.datasets.fashion_mnist
(training_images, tarining_labels),(test_images, test_labels) = fashion_minst.load_data()


training_images=training_images/255.0
test_images=test_images/255.0

model = tf.keras.Sequential([tf.keras.layers.Flatten(),
                       tf.keras.layers.Dense(512, activation=tf.nn.relu),
                       tf.keras.layers.Dense(10, activation=tf.nn.softmax)
                       ])

model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(training_images,tarining_labels,epochs=15,callbacks=[callbacks])
model.evaluate(test_images, test_labels) 

classifications = model.predict(test_images)

print(classifications[0])

print(test_labels[0])


In [ ]:
# mnist fashion dataset  solve using convolution layer

#import cv2
import numpy as np
from scipy import misc
i=misc.ascent()

import matplotlib.pyplot as plt
plt.grid(False)
plt.gray()
plt.axis('off')
plt.imshow(i)
plt.show()

i_transformed = np.copy(i)
# get the dimension's of image 
size_x = i_transformed.shape[0]
size_y = i_transformed.shape[1]

#set the filter by which we going to filter the image as per our desired
filter=[[-1, -2, -1],[0, 0, 0],[1, 2, 1]]
weight = 1

for x in range(1, size_x-1):
    for y in range(1, size_y-1):
        output_pixel = 0.0
        
        output_pixel = output_pixel + (i[x-1,y-1] * filter[0][0])
        output_pixel = output_pixel + (i[x+0,y-1] * filter[0][1])
        output_pixel = output_pixel + (i[x+1,y-1] * filter[0][2])
        
        output_pixel = output_pixel + (i[x-1,y+0] * filter[1][0])
        output_pixel = output_pixel + (i[x+0,y+0] * filter[1][1])
        output_pixel = output_pixel + (i[x+1,y+0] * filter[1][2])
        
        output_pixel = output_pixel + (i[x-1,y+1] * filter[2][0])
        output_pixel = output_pixel + (i[x+0,y+1] * filter[2][1])
        output_pixel = output_pixel + (i[x+1,y+1] * filter[2][2])
        
        output_pixel = output_pixel *weight
        
        if(output_pixel < 0):
            output_pixel = 0
        if(output_pixel > 255):
            output_pixel = 255

        i_transformed[x, y] = output_pixel
        
# plot the image
plt.gray()
plt.grid(False)
plt.imshow(i_transformed)
plt.show()

# pooling the image (compressing image )

new_x = int(size_x/2)
new_y = int(size_y/2)
newImage = np.zeros((new_x, new_y))

# iterate the from 0 to size_x with differnce of 2  
for x in range(0, size_x, 2):
    for y in range(0, size_y, 2):
        pixels = []
        
        pixels.append(i_transformed[x+0, y+0])
        pixels.append(i_transformed[x+1, y+0])
        
        pixels.append(i_transformed[x+0, y+1])
        pixels.append(i_transformed[x+1, y+1])
        
        pixels.sort(reverse =True)
        newImage[int(x/2), int(y/2)] = pixels[0]
    
# plot the compressed image

plt.gray()
plt.grid(False)
plt.imshow(newImage)
plt.show()


In [ ]:
import tensorflow as tf
#sve the dataset into one variable 
mnist = tf.keras.datasets.fashion_mnist
#load the data as per the lables
(training_images, tarining_labels),(test_images, test_labels)=mnist.load_data()

#reshape the images
training_images = training_images.reshape(60000, 28, 28, 1)
training_images=training_images/255.0

test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0

#prepare model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape = (28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),

    #add convolution layer to it
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    #flatten the output
    tf.keras.layers.Flatten(),
    
    #128 layer's
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

#Compile the code and train the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(training_images, training_lables, epochs=5)

#show the over-all  accuaracy and print the result 
test_loss,test_accuracy = model.evaluate(test_images, test_labels)
print("Test loss: {},Test accuracy: {}".format(test_loss, test_accuracy*100))

#print teh first 100 lable's
print(test_labels[:100])

#plot the image using the matplot
import matplotlib.pyplot as plt

f, axarr = plt.subplots(3, 4)

FIRST_IMAGE = 0
SECOND_IMAGE = 23
THIRD_IMAGE = 28
CONVOLUTION_NUMBER = 6 

from tensorflow.keras import models
layer_outputs = [layer.output for layer in model.layers]
activation_model = tf.keras.models.Model(inputs = model.input, outputs = layer_outputs)

for x in range(0, 4):
    
    f1 = activation_model.predict(test_images[FIRST_IMAGE].reshape(1, 28, 28, 1))[x]
    axarr[0, x].imshow(f1[0, : ,:, CONVOLUTION_NUMBER],cmap='inferno')
    axarr[0, x].grid(False)
    
    f2 = activation_model.predict(test_images[SECOND_IMAGE].reshape(1, 28, 28, 1))[x]
    axarr[1, x].imshow(f2[0, : ,:, CONVOLUTION_NUMBER],cmap='inferno')
    axarr[1, x].grid(False)
    
    f3 = activation_model.predict(test_images[THIRD_IMAGE].reshape(1, 28, 28, 1))[x]
    axarr[2, x].imshow(f3[0, : ,:, CONVOLUTION_NUMBER],cmap='inferno')
    axarr[2, x].grid(False)
    

In [ ]:
from sklearn import tree
#to create the decission tree
#data feeded is [height, weigth, shoe-size]
values=[[181, 80, 44],[177, 70, 43],[160, 60, 38],[151, 530, 44],[191, 90, 54],[154, 65, 77],[158, 80, 44],
        [185, 80, 44],[154, 80, 44],[156, 80, 45],[199, 80, 44],[178, 56, 44],[181, 90, 44],[125, 98, 35]]

community=['male', 'female','female', 'male', 'male','female','female',
           'female', 'female' ,'male', 'male', 'female','male', 'male']

create_tree = tree.DecisionTreeClassifier()
create_tree=create_tree.fit(values, community)

prediction = create_tree.predict([[208,65,60]])

print(prediction) 


In [ ]:
import numpy as np
from lightfm.datasets import fetch_movielens
from lightfm import LightFM

# fetch the data
data = fetch_movielens(min_rating = 4.0)

print(repr(data['train']))
print(repr(data['test']))

#Create Model

model = LightFM(loss = 'warp')
model.fit(data['train'], epochs = 30, num_threads = 20 )

def SampleRecomendation(model, data, user_ids):
    number_users, number_items = data['train'].shape
    
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        score = model.predict(user_id, np.arange(number_users))
        top_items = data['item_labels'][np.argsort(-score)]
        
        print("user %s"%user_id)
        print("      Know positive : ")
        
        for x in known_positives[:3]:
            print("              %s" % x)
            
        print("         Recomended :")
        
        for x in top_items[:3]:
            print("         %s" % x)

SampleRecomendation(model, data, [3, 25, 450])  

In [5]:
pip install lightFM


  Using cached https://files.pythonhosted.org/packages/e9/8e/5485ac5a8616abe1c673d1e033e2f232b4319ab95424b42499fabff2257f/lightfm-1.15.tar.gz
  Running setup.py clean for lightFM
Failed to build lightFM
  Running setup.py install for lightFM: started
    Running setup.py install for lightFM: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  ERROR: Complete output from command 'C:\Users\admin\Anaconda3\python.exe' -u -c 'import setuptools, tokenize;__file__='"'"'C:\\Users\\admin\\AppData\\Local\\Temp\\pip-install-yudhd98q\\lightFM\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\admin\AppData\Local\Temp\pip-wheel-snfu6gj4' --python-tag cp37:
  ERROR: Compiling without OpenMP support.
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.7
  creating build\lib.win-amd64-3.7\lightfm
  copying lightfm\cross_validation.py -> build\lib.win-amd64-3.7\lightfm
  copying lightfm\data.py -> build\lib.win-amd64-3.7\lightfm
  copying lightfm\evaluation.py -> build\lib.win-amd64-3.7\lightfm
  copying lightfm\lightfm.py -> build\lib.win-amd64-3.7\lightfm
  copying lightfm\_lightfm_fast.py -> build\lib.win-amd64-3.7\lightfm
  copying 

In [4]:

pip install lightFM
import numpy as np
from lightfm.datasets import fetch_movielens
from lightfm import LightFM

# fetch the data
data = fetch_movielens(min_rating = 4.0)

print(repr(data['train']))
print(repr(data['test']))

#Create Model

model = LightFM(loss = 'warp')
model.fit(data['train'], epochs = 30, num_threads = 20 )

def SampleRecomendation(model, data, user_ids):
    number_users, number_items = data['train'].shape
    
    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        score = model.predict(user_id, np.arange(number_users))
        top_items = data['item_labels'][np.argsort(-score)]
        
        print("user %s"%user_id)
        print("      Know positive : ")
        
        for x in known_positives[:3]:
            print("              %s" % x)
            
        print("         Recomended :")
        
        for x in top_items[:3]:
            print("         %s" % x)

SampleRecomendation(model, data, [3, 25, 450])  

SyntaxError: invalid syntax (<ipython-input-4-eff0e42b519b>, line 1)

In [1]:
import numpy as numpy
from functools import partial
import PIL.Image
import tensorflow as tf
import urllib.request
import os
import zipfile

In [ ]:
from tpot import TPOT
from sklearn.cross_validation  import trai
import pandas as pd
import numpy as np
